# Bienvenido al día 1 de la workshop de Nuclio Digital School: Parte 2
## 14-Enero-2025

<img src="./pictures/PANDAS_DEEP_DIVE.webp" alt="PANDAS_DD" width="700" height="300"/>

# Objetivos del notebook
---
En este notebook vamos a hacer una introducción a la librería de pandas y explorar algunas de sus ***best practices***.

**[pandas](https://pandas.pydata.org)** es una librería enfocada al análisis y manipulación de datos tabulares.<br>

Este paquete junto con [numpy](https://numpy.org), [matplotlib](https://matplotlib.org), [xgboost](https://xgboost.readthedocs.io/en/stable/) y [scikit-learn](https://scikit-learn.org/stable/index.html) constituyen las principales librerías de Machine Learning Tabular en Python.

**Dado que la mayoría de los problemas a los que se enfrentan las empresas en la vida real son problemas y datasets tabulares, aprendiendo estas 5 librerías, obtenemos una base sólida para empezar a trabajar en el mundo de Machine Learning.**

Al final del notebook, el alumno se tiene que sentir cómodo con los siguientes funcionalidades:

1. Saber como podemos "cargar" un fichero que tenemos en nuestro ordenador a pandas para poder empezar a manipularlo.
1. Entender los objetos básicos que tenemos cuando usamos pandas: **DataFrame** y **Series**.
1. Dar a conocer los métodos más simples de un pandas DataFrame: **head**, **tail**, **describe**, **info** etc.
1. Entender en que consiste el **pandas groupby** y como podemos utilizarlo para hacer cálculo de manera **rápida, limpia y eficiente.**
1. Conocer que es la **pivot_table** de pandas.
1. Ver como podemos hacer algunas visualizaciones básicas con pandas.
---

# Let's go!

---

<a id='index'></a>
## Índice

[Imports del notebook](#imports_notebook)<br>

## Dataset
[Dataset que vamos a utilizar: animes](#animes)<br>


## Intro to pandas
[Helpers pandas](#helpers_pandas)<br>
[¿Que es el pandas groupby?](#pandas_groupby)<br>
[Pandas transform (**CORE IDEA**)](#pandas_transform)<br>
[Groupby eficiente (**CORE IDEA**)](#pandas_gbe)<br>
[Funciones personalizadas](#pandas_custom_func)<br>
[Nueva columna utilizando la información de 2 columnas](#pandas_custom_func_2_cols)<br>

## Conclusión
[Conclusión](#conclusion)<br>

## Referencias
[Referencias y lecturas recomendables](#referencias)<br>

<a id='imports_notebook'></a>
# Imports del notebook
[Volver al índice](#index)

En este apartado hacemos los principales imports del notebook.<br>
Sobre todo vamos a trabajar con **pandas** y **matplotlib**.

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print("Working with these versions of libraries\n")
print("-"*50)
print(f"Pandas version {pd.__version__}")

Working with these versions of libraries

--------------------------------------------------
Pandas version 2.2.1


<a id='animes'></a>
# Dataset que vamos a utilizar: animes
[Volver al índice](#index)

Todo proyecto de Machine Learning requiere de datos. Los datos son la parte **FUNDAMENTAL** y si la calidad de los datos es mala, muchas veces los resultados que vamos a obtener van a ser malos.

Gran parte de un proyecto de ML consiste en limpiar, analizar y preparar los datos para **"alimentarlo"** al algoritmo. Muy a menudo, todo esto se realiza con **pandas**.

Dado que nuestro objetivo final es crear un recomendador basado en el historial de los clientes (lo haremos en el notebook [CBRS.ipynb](./CBRS.ipynb)), en este notebook vamos a hacer la limpieza y el análisis de los datos y por el camino vamos a aprender de pandas.

**¿Que dataset vamos a utilizar en nuestro proyecto?**

Vamos a utilizar un dataset muy famoso de [Animes Japoneses](https://es.wikipedia.org/wiki/Anime) que se puede descargar de manera gratuita del [siguiente enlace](https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database/data).

# Intro to pandas

<a id='helpers_pandas'></a>
# Cargar datos a pandas
[Volver al índice](#index)

Normalmente los datos con los que trabajamos en la vida real están en diferentes bases de datos y utilizamos el lenguaje de programación [SQL](https://es.wikipedia.org/wiki/SQL) para descargarlos.

En este workshop, para simplificar algo las cosas, vamos a trabajar con ficheros que están en el disco duro de nuestro ordenador para "leerlos" con pandas.

In [15]:
# hacer el import de pandas
import pandas as pd

In [20]:
# donde están mis datos
! pwd

/Users/nicolaepopescul/code/nuclio_charlas


In [23]:
# que tengo en esta ruta
! ls

CBRS.ipynb   INTRO.ipynb  PANDAS.ipynb README.md    input        pictures


In [24]:
# que tengo en la carpeta de input
! ls /Users/nicolaepopescul/code/nuclio_charlas/input

cf_anime.csv           cf_anime.pkl           cf_rating.parquet.gzip
cf_anime.parquet.gzip  cf_anime.xlsx


In [40]:
# leemos un fichero en un pandas DataFrame
df = pd.read_parquet("./input/cf_anime.parquet.gzip")

In [42]:
df = df.set_index("anime_id")

Acabamos de cargar nuestro fichero en un pandas **DataFrame.**

El **DataFrame** es el objeto central en pandas y representa una tabla/matriz que tiene 2 ejes.

Un eje son las **filas del DataFrame** y básicamente vienen a representar los registros que tenemos en la tabla. A menudo, cuando trabajamos en problemas de ML Supervisados, también los llamamos "instancias".

El otro eje que tenemos son **las columnas del DataFrame**. Las columnas a menudo se conocen como las "features" o "atributos" y vienen a captar diferentes variables/información sobre nuestras instancias.

Por ejemplo: si 1 fila es un cliente, las columnas son las variables que tenemos de este cliente y podrían ser la edad, el patrimonio, el móvil que usa etc.

Con el método de `df.shape` podemos llegar a saber el número de filas y de columnas que tiene nuestro dataset.

In [43]:
df.shape

(12294, 6)

Cuando hacemos nuestro primer contacto con un dataset, a menudo nos interesa ver los primeros 5 registros o los últimos 5 registros de una tabla.

Los métodos de `df.head()` o `df.tail()` sirven justo para este propósito.

In [44]:
df.head()

,name,genre,type,episodes,rating,members
anime_id,,,,,,
32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [45]:
df.tail(5)

,name,genre,type,episodes,rating,members
anime_id,,,,,,
9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
5543,Under World,Hentai,OVA,1,4.28,183
5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175
26081,Yasuji no Pornorama: Yacchimae!!,Hentai,Movie,1,5.46,142


Muy a menudo queremos sacar una pequeña muestra de clientes de nuestra tabla para analizarla rápidamente.

El método `df.sample(nr_registros)` nos permite hacer una muestra aleatoria sin reemplazo.

Con la siguiente línea de código vamos a seleccionar al azar 25 registros de nuestro dataset.

In [46]:
df.sample(25)

,name,genre,type,episodes,rating,members
anime_id,,,,,,
4800,Huckleberry Finn Monogatari,Adventure,TV,26,6.53,665
31361,Neko Enikki,"Comedy, Slice of Life",TV,13,6.67,133
17733,Dibetagurashi: Ahiru no Seikatsu,"Comedy, Slice of Life",TV,39,5.99,586
33803,Sengoku Choujuu Giga,"Demons, Historical, Supernatural",TV,26,5.56,3248
33505,Tsumamigui 3 The Animation,Hentai,OVA,Unknown,6.87,1400
1397,Macross 7,"Action, Adventure, Comedy, Drama, Mecha, Milit...",TV,49,7.23,15420
16377,Peeping Life: The Perfect Extension,"Comedy, Slice of Life",OVA,10,5.89,245
11673,Hal no Fue,"Adventure, Kids",Movie,1,6.44,202
15359,Kyousou Giga (2012),"Action, Fantasy, Supernatural",ONA,5,7.25,17584


Vemos que nuestro dataset está formado por 6 columnas.

Cuando usamos pandas, decimos que 1 columna es un **pandas Series**. 

Por tanto, nuestro **DataFrame** es básicamente una colección de 6 pandas **Series**.

Si queremos seleccionar 1 única columna de nuestro **DataFrame** lo podemos hacer con el siguiente código.

In [36]:
series = df["rating"]

Las **Series** a menudo tienen los mismos métodos que un **DataFrame**: `head`, `tail`etc.

In [37]:
series.head()

0    9.37
1    9.26
2    9.25
3    9.17
4    9.16
Name: rating, dtype: float64

En el ejemplo de antes, hemos utilizado el nombre de la columna entre **corchetes** para seleccionar 1 columna.

Existen otras formas de hacer lo mismo.

Por ejemplo, si el nombre **<u>no contiene ningún espacio</u>**, podemos escribir `df.nombre_columna` para obtener este **Series**.

In [38]:
df.rating

0        9.37
1        9.26
2        9.25
3        9.17
4        9.16
         ... 
12289    4.15
12290    4.28
12291    4.88
12292    4.98
12293    5.46
Name: rating, Length: 12294, dtype: float64

Por último, es muy común utilizar el método `loc` o `iloc` para seleccionar columnas o filas.

Con `loc` podemos acceder a diferentes filas en base al nombre de la fila o columna.

In [53]:
df.head()

,name,genre,type,episodes,rating,members
anime_id,,,,,,
32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [62]:
# en este ejemplo, 32281 es el anime_id que tenemos en la primera fila.
# si fuese un text en vez de número deberíamos escribir el nombre con comillas simples '' o comillas dobles "".
df.loc[32281]

name                              Kimi no Na wa.
genre       Drama, Romance, School, Supernatural
type                                       Movie
episodes                                       1
rating                                      9.37
members                                   200630
Name: 32281, dtype: object

Con el loc puedo seleccionar diferentes filas y columnas especificando más parámetros en el loc.

In [66]:
df.loc[32281, "genre"]

'Drama, Romance, School, Supernatural'

Si quiero seleccionar "genre" y también la columna de "type" lo puedo hacer poniendo "genre" y "type" dentro de una lista de python: `["genre", "type"]` es decir, usamos otra vez corchetes.

In [67]:
df.loc[32281, ["genre", "type"]]

genre    Drama, Romance, School, Supernatural
type                                    Movie
Name: 32281, dtype: object

Mientras que `loc` permite acceder a las diferentes filas y columnas por el nombre, `iloc` lo hace por el índice.

**Nota:** en la mayoría de los lenguajes de programación, el índice empieza por el 0. Es decir, el primer eleme noto tiene el índice de 0, el segundo elemento tiene el índice de 1 etc.

In [68]:
df.head()

,name,genre,type,episodes,rating,members
anime_id,,,,,,
32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [70]:
df.iloc[0]

name                              Kimi no Na wa.
genre       Drama, Romance, School, Supernatural
type                                       Movie
episodes                                       1
rating                                      9.37
members                                   200630
Name: 32281, dtype: object

In [71]:
df.iloc[0, 1]

'Drama, Romance, School, Supernatural'

In [73]:
df.iloc[0, [1, 2]]

genre    Drama, Romance, School, Supernatural
type                                    Movie
Name: 32281, dtype: object

In [ ]:
def get_category(nr_rows):
    '''
    Generates random category.
    '''
    categories = ["A", "B", "C", "X", "Y", "Z"]
    random_categories = np.random.choice(categories, size = nr_rows)
    
    return random_categories

In [ ]:
def generate_dummy_dataframe(nr_rows, nr_columns, nr_categoricals):
    '''
    Generates a dummy DataFrame.
    '''
    la = []
    
    for nr_ in range(nr_columns):
        la.append(np.random.randint(-1000, 1000, size = nr_rows))
            
    df = pd.DataFrame(la).T
    
    cat_index = [cat + str(i) for i, cat in enumerate(get_category(nr_rows = nr_rows))]
    df.index = cat_index
    
    dummy_column_names = ["Column_{}".format(i) for i in range(1, nr_columns + 1)]
    
    df.columns = dummy_column_names
    
    for nr_categorical in range(nr_categoricals):
        df["Categorical_{}".format(nr_categorical + 1)] = get_category(nr_rows = nr_rows)
    
    return df
    

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH, "pd_sklearn_dummy.csv"))

Veamos nuestro dataframe

In [ ]:
df.head(5)

## ¿Que es el pandas groupby?
<a id='pandas_groupby'></a>
[Volver al índice](#index)<br>

La gran mayoría de los ejemplos que siguen a continuación, se van a basar en el **pandas groupby**.<br>
Por este motivo, es de vital importancia entender que hay **"dentro"** de un pandas groupby y saber cuando es el momento de usarlo.

Veamos un ejemplo práctico con nuestro dataset.

### Pregunta 1: hacer la suma de Column_1 cuando Categorical_1 == "B"

In [ ]:
# code goes here

In [ ]:
# posible solución
df[df["Categorical_1"] == "B"]["Column_1"].sum()

Cuando queremos hacer un cálculo rápido y sencillo, podemos utilizar los **filtros de pandas** para hacer una selección de un subconjunto de datos y aplicar nuestra fórmula.

No obstante, ¿Que pasaría si necesitamos calcular esta suma para todas las categorías que hay en Categorical_1?

Una implementación rápida sería utilizar una for loop, como en el siguiente ejemplo.

In [ ]:
# posible solución
result_dict = {}

for category in sorted(df["Categorical_1"].unique()):
    result_dict[category] = df[df["Categorical_1"] == category]["Column_1"].sum()
    
result_dict

La implementación anterior es válida, pero hay un forma mucho mejor de hacerlo. Utilizar el pandas groupby.

In [ ]:
# creamos un objeto de pandas groupby y lo asignamos a gb_df
gb_df = df.groupby("Categorical_1")
gb_df

Cuando montamos un pandas groupby e inspeccionamos el objetvo, vemos que el resultado es algo similar a esto:
#### **<pandas.core.groupby.generic.DataFrameGroupBy object at 0x7fe5c81de430>**

Lo que obtenemos no es más que la dirección en memoria del pandas groupby (en nuestro caso, gb_df).

Ahora bien, una forma muy sencilla para ver el contenido del gb_df es utilizar el método **get_group**. Al método get_group le debemos pasar una tupla. El tamaño de la tupla debe ser igual al número de columnas por las cuales has hecho el groupby.

Por ejemplo:<br>

---

**Agrupamos por 2 columnas, la tupla debe ser de 2 elementos**

gb_df = df.groupby(["Cat1", "Cat2"])<br>
gb_df.get_group(**(e1, e2)**)

**Agrupamos por 3 columnas, la tupla debe ser de 3 elementos**

gb_df = df.groupby(["Cat1", "Cat2", "Cat3"])<br>
gb_df.get_group(**(e1, e2, e3)**)

Donde e1, e2 y e3 son elemento 1, elemento 2 y elemento 3 y son categorías que existen en la Columna Cat1, Cat2 y Cat3 respectivamente (e1 existe en Cat1, e2 existe en Cat2 y e3 existe en Cat3).

---

### La única condición que deben cumplir los elementos de la tupla, es ser categorías que existen dentro de las columnas del groupby.

Veamos unos ejemplos con nuestro dataframe.

In [ ]:
# Hice el groupby por Categorical_1, la categoría A existe dentro de Categorical_1
# Por este motivo, cuando hago get_group(("A")) veo que todos los elementos de Categorical_1 == "A"
gb_df.get_group(("A")).head()

In [ ]:
# lo mismo pero con el grupo C
gb_df.get_group(("C")).head()

In [ ]:
# que pasaría si intento buscar el grupo I
try:
    gb_df.get_group(("I")).head()
except:
    print('El gb_df.get_group(("I")).head() ha fallado! :(')

¡La ejecución anterior nos ha dado un error! Intenté buscar una categoría que no existe.

### (CORE IDEA) Podemos pensar en un groupby como un conjunto de dataframes pequeños (donde las categorías siempre coinciden). Por tanto, puedo aplicar la mayoría de las transformaciones propias de un df.<br>
De hecho, hacer el get_group es idéntico a hacer un filtro de pandas groupby.

In [ ]:
df[df["Categorical_1"] == "A"].head()

In [ ]:
gb_df.get_group(("A")).head()

In [ ]:
df[df["Categorical_1"] == "A"].equals(gb_df.get_group(("A"))) # comprobamos que son iguales

<a id='tip_1_pandas_gb_index'></a>
### Si nos fijamos de manera detenida, vemos que el index del df filtrado, es el mismo que el index del get_group.

Esta parte va a ser muy importante cuando vamos a trabajar con **pandas transform**.

De momento, nos quedamos con la idea de que hacer un filtro o get_group de groupby **me permite acceder a un subconjunto del dataframe en cuestión.**

In [ ]:
print(result_dict)

In [ ]:
# best praxis
gb_df["Column_1"].sum().to_dict()

### Pandas transform (CORE IDEA)
<a id='pandas_transform'></a>
[Volver al índice](#index)<br>

### Pregunta 2: añadir una nueva columna al df que sea la suma total de Column_1 respetando cada categoría de Categorical_1

In [ ]:
# code goes here

In [ ]:
# posible solución
result_dict_gb = df.groupby("Categorical_1")["Column_1"].sum().to_dict()

df["New_col_1"] = df["Categorical_1"].map(result_dict_gb)

df

La implementación anterior utiliza una variable temporal, en nuestro caso result_dict_gb para guardar los resultados y después mappear el resultado al df.  Puede parecer una buena forma de hacerlo hasta que conocemos al pandas **transform**.

In [ ]:
# best praxis
df["New_col_1_bp"] = df.groupby("Categorical_1")["Column_1"].transform(np.sum)

Pandas **transform** es una mejor solución porque permite conseguir el mismo resultado
en 1 única línea de código. No hay necesidad de guardar nada de manera temporal. Con esto conseguimos un código más limpio y elegante.

Internamente dentro de groupby lo que ocurre es:
1. Calculamos la suma de Column_1 para cada categoría.
2. Hacemos el merge con el df orignal, por el index del dataframe.

[¿Os acordáis cuando decíamos que dentro del get_group se preserva el index original?](#tip_1_pandas_gb_index)

Si nos fijamos en la siguiente línea de código, el index es idéntico al df original.

In [ ]:
df.groupby("Categorical_1")["Column_1"].transform(np.sum).head()

In [ ]:
df.head(5)

### Groupby eficiente (CORE IDEA)
<a id='pandas_gbe'></a>
[Volver al índice](#index)<br>

En más de una ocasión, queremos hacer varios cálculos dentro de un pandas groupby (por ejemplo: hacer diferentes funciones de agregación como la suma, media, máximos sobre diferentes columnas) y después **concatenar** estos resultados en un nuevo dataframe. Hacemos esto a menudo durante el **EDA** en un proyecto de Machine Learning.

---
Habitualmente el patrón que se sigue es:

1. variable_1 = df.groupby(["CAT_1", "CAT_2"])["COLUMNA_1"].funcion_agregación()
2. variable_2 = df.groupby(["CAT_1", "CAT_2"])["COLUMNA_2"].otra_funcion_agregación()
3. df_resumen = pd.concat([variable_1, variable_2], axis = 1)

### Pregunta 3: crear un df_resumen que contenga la suma, media (para Column_1) y un contador de los valores positivos para la columna Column_2, agrupados por Categorical_1

El resultado que nos debería dar un df similar a este.

In [ ]:
result_data_dict = {
    'Sum_column_1': {'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769},
    'Mean_column_1': {'A': 10.57, 'B': 5.78, 'C': -9.27, 'X': 78.75, 'Y': 35.59, 'Z': 67.84},
    'Counter_non_negative_values_column_2': {'A': 101, 'B': 85, 'C': 89, 'X': 77, 'Y': 91,'Z': 73}
}

In [ ]:
pd.DataFrame(data = result_data_dict)

In [ ]:
# code goes here

In [ ]:
# posible solución
suma_column_1_categorical_1 = df.groupby("Categorical_1")["Column_1"].sum()
mean_column_1_categorical_1 = df.groupby("Categorical_1")["Column_1"].mean()
positive_counter = df.groupby("Categorical_1")["Column_2"].agg(lambda series: len(series[series >= 0]))

df_resumen = pd.concat([
    suma_column_1_categorical_1,
    mean_column_1_categorical_1,
    positive_counter
], axis = 1)

df_resumen.columns = ["Sum_column_1", "Mean_column_1", "Counter_non_negative_values_column_2"]

del suma_column_1_categorical_1, mean_column_1_categorical_1, positive_counter

df_resumen.round(2)

Si miramos de manera crítica la celda anterior, podemos ver las siguientes ineficiencias:
1. Generamos 3 variables temporales que no nos hacen falta a posteriori.
2. Debemos añadir varias líneas de código para concatenar estos resultados.
3. Tenemos que hacer limpieza de las variables temporales (buena praxis).
4. Debemos renombrar las columnas del df_resumen.

Todos los pasos anteriores para conseguir un df_resumen.

Veamos una forma mucho más elegante y eficiente de hacerlo.

In [ ]:
# best praxis
df.groupby("Categorical_1").agg(
    Sum_column_1 = ("Column_1", np.sum),
    Max_column_1 = ("Column_1", np.max),
    Mean_column_1 = ("Column_1", np.mean),
    Counter_non_negative_values_column_2 = ("Column_2", lambda series: len(series[series >= 0])),
    Sum_non_negative_values_column_2 = ("Column_2", lambda series: np.sum(series[series >= 0])),
).round(2)

Conseguimos el mismo resultado que antes, pero de una manera más rápida y elegante. 

**En realidad, hacemos 2 operaciones adicionales y todo con menos líneas de código.**

### ¿Todos los alumnos has conseguido sacar el resultado Counter_non_negative_values_column_2?

### Funciones personalizadas
<a id='pandas_custom_func'></a>
[Volver al índice](#index)<br>

En el ejemplo anterior, para calcular **Counter_non_negative_values_column_2** tuvimos que crear una función personalizada (custom function) en forma de ***lambda***.

Esto es de extrema relevancia porque significa que dentro de un **pandas groupby** yo puedo utilizar no solo las funciones tipo: suma, media, máximo etc. sino **cualquier función siempre y cuando haga un cálculo y devuelva un resultado agregado.** De hecho, el resultado podría hasta ser un string. 

Veamos algunos ejemplos.

In [ ]:
def custom_func_non_negative_sum(series):
    '''
    Sums non negative values from a pandas series
    '''
    return np.sum(series[series >= 0])

La función anterior es idéntica a la siguiente función anónima **lambda series: np.sum(series[series >= 0])**

In [ ]:
df.groupby("Categorical_1")["Column_2"].agg(custom_func_non_negative_sum)

Por supuesto la función anterior la podemos utilizar con un pandas transform

In [ ]:
df["New_col_2_cf_bp"] = df.groupby("Categorical_1")["Column_2"].transform(custom_func_non_negative_sum)

In [ ]:
df.head(1)

Antes hemos comentado, que la función de agregación customizada puede hasta devolver un string como resultado.

Veamos un ejemplo rápido.

In [ ]:
def custom_func_returns_string(series):
    '''
    Custom function that returns a string as a result
    '''
    positive_values = len(series[series >= 0])
    negative_values = len(series[series < 0])
    
    return f"Positive Values: {positive_values} Negative Values: {negative_values}"

In [ ]:
print(df.groupby("Categorical_1")["Column_1"].agg(custom_func_returns_string))

In [ ]:
df["New_col_2_cfs_bp"] = df.groupby("Categorical_1")["Column_1"].transform(custom_func_returns_string)

In [ ]:
df.sample(5)

En la práctica no obstante, usamos este tipo de funciones sólo cuando queremos hacer un reporting sobre un dataframe e incluirlo en un informe final.

### Nueva columna utilizando la información de 2 columnas
<a id='pandas_custom_func_2_cols'></a>
[Volver al índice](#index)<br>

Vamos a darle otra vuelta de tuerca a las funciones personalizadas.

Si hacemos un breve recap de lo visto hasta ahora, sabemos que:
1. Dentro de un groupby hay un montón de dfs "pequeños". Estos dataframes no se solapan con otros porque tienen categorías únicas (en función de las columnas por las cuales se ha hecho el groupby).
2. De alguna manera, hacer un groupby y llamar get_group (que es lo que ocurre dentro de un groupby pero para todas las categorías) es similar a hacer un filtro de df.
3. Al hacer un filtro o groupby, sigo operando todo el rato con un df. Si sigo operando con un df, puedo acceder a cualquier columna del df.
4. Por tanto, puedo realizar una función de agregación teniendo en cuenta la información de más de una columna, sin necesidad de **iterar** sobre el df.

Veamos un ejemplo sencillo:

### Pregunta 4: calcular la media de la Column_1 cuando tengo en la Column_2 más valores positivos que negativos (len_positivos >= len_negativos), en caso contrario devolver -1 todo esto dentro de un groupby.

In [ ]:
# code goes here

In [ ]:
# posible solución
d_ = {}

for category in sorted(df["Categorical_1"].unique()):

    sample_df = df[
        (df["Categorical_1"] == category)
    ]

    value_counts = (sample_df["Column_2"] >= 0).value_counts()
    
    true_values = value_counts.loc[True]
    false_values = value_counts.loc[False]
    
    if true_values >= false_values:
        d_[category] = sample_df["Column_1"].mean()
    else:
        d_[category] = -1

d_

In [ ]:
# best praxis
def custom_agg_function_2_cols(df):
    '''
    Custom function that uses values from 1 column to calculate the value for another column.
    '''
    value_counts = (df["Column_2"] >= 0).value_counts()
    true_values = value_counts.loc[True]
    false_values = value_counts.loc[False]
    
    if true_values >= false_values:
        return df["Column_1"].mean()
    else:
        return -1

df.groupby("Categorical_1").apply(custom_agg_function_2_cols)

### Conclusión
<a id='conclusion'></a>
[Volver al índice](#index)<br>

A lo largo de este notebook hemos aprendido nuevas técnicas y algunas de las best practices de **pandas** y **sklearn**.

### Siempre debemos intentar escribir un código legible, limpio y procurar reutilizar los módulos o paquetes ya disponibles (para no reinventar la rueda).

Tanto **pandas como sklearn** ofrecen mucha flexibilidad así como un montón de funcionalidades para nuestro día a día.

La utilización eficiente de pandas groupby puede ayudar mucho a la hora de hacer un análisis más rápido y conciso.

A su vez sklearn con sus Transformers, Pipelines y Cross Validation nos da herramientas muy potentes para entrenar nuestros modelos de ML.

Recomendamos a los alumnos que los vayan incorporando en su kit diario cuanto antes.

# El alumno puede probar montar un nuevo pipeline con todas las columnas del dataset para practicar lo aprendido.

### Referencias y lecturas recomendables
<a id='referencias'></a>
[Volver al índice](#index)<br>

A continuación dejamos algunos links útiles para profundizar en algunos de los conceptos que hemos visto en el notebook:

[Sitio oficial de pandas](https://pandas.pydata.org/)

[Apply vs Transform en pandas groupby](https://stackoverflow.com/questions/27517425/apply-vs-transform-on-a-group-object/47143056#47143056)

[Sitio oficial de sklearn](https://scikit-learn.org/stable/)

[Diseño de sklearn](https://arxiv.org/pdf/1309.0238.pdf)

[Model Evaluation, Model Selection, and Algorithm Selection in Machine Learning (MUY RECOMENDABLE)
](https://arxiv.org/pdf/1811.12808.pdf)

[Diferentes estrategias de Cross Validation](https://scikit-learn.org/stable/modules/cross_validation.html)

[Duck Typing](https://en.wikipedia.org/wiki/Duck_typing)